In [1]:
# Imports

import numpy as np
import pandas as pd

from random import shuffle

from func_matching import calculate_match
from func_args import get_args

# eliminar
# from func_grafos import create_graph # , get_best_match eliminar
from func_aquisition import adquisition_survey
from func_outputs import outputs
from func_normalize import *


def least_common(queue):
    ''' Ordenar por menos número de entrevistas en la cola '''
    res = sorted(queue, key=queue.get, reverse=False)
    return res

def get_next_interview(lista_matching_ordenada, used, students_dict_queue): 
    
    students_ordered = least_common(students_dict_queue)
    # print('````````````````````````````')
    # print('students_ordered: ', students_ordered)
    # print('used: ', used)
    for student in students_ordered: # según el orden de los menos entrevistados
        if student not in used: 
            for match in lista_matching_ordenada:
                if match['company'] not in used and student == match['student']: # TO DO all() 
                    
                    # print('used: ', used)
                    # print(student)
                    # print(match['company'])
                    # print('match', match, '\n')
                    return match
    return None
    
def get_rondas(lista_matching, n_rondas, students, companies): 
    lista_matching_ordenada = sorted(lista_matching, key = lambda x: x['weight'], reverse=True)
    students_dict_queue = generate_student_interviews(students)
    
    res = []
    mesas = len(companies)
    for ronda in range(n_rondas): # rondas
        used = []
        
        for mesa in range(mesas): # mesas
            
            m = get_next_interview(lista_matching_ordenada, used, students_dict_queue)
            # print(m)
            if m == None: 
                break
            student, company, w = m['student'], m['company'], m['weight']
            # print('pre: ',used)                        
            used.extend([student, company]) # añadimos empresa y compañia para que no vuelvan a aparecer en esta ronda
            # print('post:', used)
            lista_matching_ordenada.remove(m) # eliminamos el matching para que no se repita

            res.append({'ronda': ronda, 'company': company, 'student': student, 'weight': w})
            # print(students_dict_queue[student])
            # print(students_dict_queue.get(student))
            students_dict_queue[student] += 1
            # print(students_dict_queue.get(student))
            # print(students_dict_queue[student])

    return res    

def generate_student_interviews(students_list): 
    '''
    Para contar el número de entrevistas que lleva hechas un alumno
    '''
    return {student: 0 for student in students_list}


def main(): 
    # Captura de argumentos. 
    bootcamp, companies_filename, students_filename, n_rondas = get_args()
    
    # DFs limpios
    df_companies, students = adquisition_survey(bootcamp, companies_filename, students_filename)
    
    # Normalize
    students, companies = normalize_2dfs(students, df_companies)

    # TO DO: crear la tabla matching
    lista_matching = calculate_match(students, df_companies, bootcamp)
    
    rondas = get_rondas(lista_matching, n_rondas, students.index, companies.index)
    
    outputs(rondas)


if __name__== '__main__':
    main()

